In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from cleaner import DataCleaner
from utils import Utils
from preprocessing import Processor

In [3]:
util = Utils()
cleaner = DataCleaner()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\App

In [ ]:
# Get URL from DVC
train_store = 'data/relation_dev.txt'
test_store = 'data/relation_test.txt'
repo = 'https://github.com/HenokD11/SalesPrediction'
version = '8a89e3b46e60a61e908a32e6235255fc'

In [ ]:
news_df=util.read_from_dvc(train_store, repo,version,low_memory=False)
news_df=news_df.sample(frac=1)

In [16]:
train_news=news_df.head()
test_news=news_df.tail()
train_news

,Domain,Title,Description,Body,Link,timestamp,Analyst_Average_Score,Analyst_Rank,Reference_Final_Score
0,rassegnastampa.news,Boris Johnson using a taxpayer-funded jet for ...,…often trigger a protest vote that can upset…t...,Boris Johnson using a taxpayer-funded jet for ...,https://rassegnastampa.news/boris-johnson-usin...,2021-09-09T18:17:46.258006,0.0,4,1.96
1,twitter.com,"Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...",http://twitter.com/CoruscaKhaya/status/1435585...,2021-09-08T13:02:45.802298,0.0,4,12.00
2,atpe-tchad.info,Marché Résines dans les peintures et revêtemen...,…COVID-19…COVID…COVID…COVID-19 et Post COVID…C...,Le rapport d’étude de marché Résines dans les ...,http://atpe-tchad.info/2021/09/13/marche-resin...,2021-09-13T07:32:46.244403,0.0,4,0.05
3,badbluetech.bitnamiapp.com,"AI drives data analytics surge, study finds",…hate raiders' linked to automated harassment ...,How to drive the funnel through content market...,http://badbluetech.bitnamiapp.com/p.php?sid=21...,2021-09-11T00:17:45.962605,0.0,4,6.10
4,kryptogazette.com,Triacetin Vertrieb Markt 2021: Globale Unterne...,…Abschnitten und Endanwendungen / Organisation...,Global Triacetin Vertrieb-Markt 2021 von Herst...,https://kryptogazette.com/2021/09/08/triacetin...,2021-09-08T12:47:46.078369,0.0,4,0.13


In [17]:
test_news

,Domain,Title,Description,Body,Link,timestamp,Analyst_Average_Score,Analyst_Rank,Reference_Final_Score
5,mype za,male arrest murder elderli femal cofimvaba sap...,crime stamp n1 r101 road appear court sap crim...,south african polic servic offic provinci comm...,,2021 09 10t00 17 46 055622,1.33,2,11.00
6,eminetra za,7th anniversari scoan collaps nigeria sabc news,lago nigeria 84 south african kill build hous ...,today 7th anniversari tragic collaps synagogu ...,,2021 09 12t05 17 50 279081,0.00,4,10.10
7,eminetra za,construct sector expect boost riot loot repair,addit spend build repair secur kwazulu natal h...,construct activ grow steadili 4 second quarter...,,2021 09 09t09 02 46 320793,1.66,1,1.36
8,news24 com,news24 com court dismiss attempt eskom ceo pun...,lawsuit public particip design intimid johanne...,eskom ceo matshela mose koko seek r500 000 dam...,,2021 09 09t19 32 46 239682,0.33,3,2.40
9,manometcurr com,global region beta caroten market research 202...,key player dsm basf alli biotech chr histor ma...,global region beta caroten market research 202...,,2021 09 13t03 02 45 609228,0.00,4,0.22


In [7]:
full_processed_df=processor.prepare_text(train_news.copy())
description_processed=processor.prepare_text(train_news.copy(),columns=["Description"])[["Description","Analyst_Average_Score"]]
title_processed=processor.prepare_text(train_news.copy(),columns=["Title"])[["Title","Analyst_Average_Score"]]
body_processed=processor.prepare_text(train_news.copy(),columns=["Body"])[["Body","Analyst_Average_Score"]]
unprocessed=train_news

processed_test=processor.prepare_text(test_news)

In [8]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)

In [9]:
full_promp_trainer=full_promp_trainer.reset_index().drop(['index'],axis=1)

In [10]:
full_promp_trainer=full_promp_trainer[['Domain','Title','Description','Body','timestamp','Analyst_Average_Score']]
full_promp_trainer.rename(columns={"Analyst_Average_Score":"Analyst Average Score"},inplace=True)
# full_promp_trainer=cleaner.convert_to_datetime(full_promp_trainer,['timestamp'])
full_promp_trainer=full_promp_trainer.fillna('')
full_promp_trainer.head()

,Domain,Title,Description,Body,timestamp,Analyst Average Score
0,rassegnastampa news,bori johnson taxpay fund jet elect campaign fi...,trigger protest vote upset minist breach minis...,bori johnson taxpay fund jet elect campaign fi...,2021 09 09t18 17 46 258006,0.0
1,twitter com,stumbl interest case woman face evict home inh...,stumbl interest case woman face evict home inh...,stumbl interest case woman face evict home inh...,2021 09 08t13 02 45 802298,0.0
2,atp tchad info,marché résine dan le peintur et revêtement 202...,covid 19 covid covid covid 19 et post covid co...,le rapport d étude marché résine dan le peintu...,2021 09 13t07 32 46 244403,0.0
3,badbluetech bitnamiapp com,ai drive data analyt surg studi find,hate raider link autom harass campaign engadge...,drive funnel content market link build searche...,2021 09 11t00 17 45 962605,0.0
4,kryptogazett com,triacetin vertrieb markt 2021 global unternehm...,abschnitten und endanwendungen organisationen ...,global triacetin vertrieb markt 2021 von herst...,2021 09 08t12 47 46 078369,0.0


In [11]:
processor.prepare_tuner(full_promp_trainer)

tuner prepared successfuly


In [12]:
# uncomment these for first time use
# full_processed_df.to_csv("../data/trainer_news.csv",index=False)
# processed_test.to_csv("../data/test_news.csv",index=False)